In [1]:
import pandas as pd
import datetime
import numpy as np
import argparse
import math
from pysplit.trajectory_generator import generate_bulktraj

In [8]:
parser = argparse.ArgumentParser(description='Calculating hysplit trajs.')

parser.add_argument("--climate", choices=["hist", "pgw"], required=True, type=str, help="This is the hist or pgw choice.")
parser.add_argument("--month", required=True, type=int, help="Storm month for trajectory calculation.")
parser.add_argument("--ens", required=True, type=int, help="Ensemble number.")

args=parser.parse_args()

usage: ipykernel_launcher.py [-h] --climate {hist,pgw} --month MONTH --ens ENS
ipykernel_launcher.py: error: the following arguments are required: --climate, --month, --ens


SystemExit: 2

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
which_climate='pgw'
which_month=5
ens_number=1

In [3]:
work_help1=np.hstack([np.array([0 for i in range(36)]),
                      np.array([1 for i in range(36)]),
                      np.array([2 for i in range(36)]),
                      np.array([3 for i in range(36)]),
                      np.array([4 for i in range(36)]),
                      np.array([5 for i in range(36)]),
                      np.array([6 for i in range(36)]),
                      np.array([7 for i in range(36)]),
                      np.array([8 for i in range(36)]),
                      np.array([9 for i in range(36)])
                     ])

work_help2=np.hstack([np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)]),
                      np.array([i+1 for i in range(36)])
                     ])

In [4]:
which_working=f"work{work_help2[36]}_{work_help1[36]}"

In [5]:
print(which_working)
print(which_climate)
print(which_month)
print(ens_number)

work1_1
hist
5
1


In [185]:
#which_climate='hist'

In [11]:
csv_file=pd.read_csv(f'/glade/work/bpoujol/Moisture_tracking/trajectory_information_{which_climate}.csv')

dates=[]
for datetime_string in csv_file['TIME (UTC)'].values:
    dates.append(datetime.datetime.strptime(datetime_string, '%Y-%m-%d_%H:%M:%S'))
ready_dates=pd.to_datetime(np.array(dates))

csv_file['YEAR']=ready_dates.year
csv_file['MONTH']=ready_dates.month
csv_file['DAY']=ready_dates.day
csv_file['HOUR']=ready_dates.hour

csv_file=csv_file[csv_file['MONTH']==which_month]

In [12]:
csv_file

,TRAJ. ID,STORM ID,SUBREGION,TIME (UTC),LAT,LON,PBL HEIGHT (M),TIME (INTEGER),YEAR,MONTH,DAY,HOUR
114,115,200405004,3,2004-05-25_00:00:00,63.055416,-151.66118,235.71670,576,2004,5,25,0
115,116,200405004,3,2004-05-25_06:00:00,63.013798,-152.05356,170.06580,582,2004,5,25,6
116,117,200405007,6,2004-05-25_18:00:00,64.168970,-141.12325,593.63350,594,2004,5,25,18
117,118,200405007,6,2004-05-26_00:00:00,64.078630,-141.40540,648.24910,600,2004,5,26,0
118,119,200405012,6,2004-05-09_06:00:00,63.139896,-145.79060,105.67163,198,2004,5,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1599,1600,201505038,4,2015-05-29_06:00:00,68.544480,-154.50528,154.14917,678,2015,5,29,6
1600,1601,201505038,4,2015-05-29_12:00:00,68.771790,-154.25055,253.44025,684,2015,5,29,12
1601,1602,201505044,5,2015-05-30_00:00:00,68.852390,-142.87631,203.17159,696,2015,5,30,0
1602,1603,201505045,6,2015-05-09_00:00:00,61.731930,-143.21198,76.19762,192,2015,5,9,0


In [8]:
def ens_create(ens_num, lat, lon):
    
    """Extract the ensemble member's lat and lon coordinates.
    
    """
    ens_help=np.hstack([np.array([i for i in range(49)]),
                        np.array([i for i in range(49)]),
                        np.array([i for i in range(49)]),
                        np.array([i for i in range(49)]),
                        np.array([i for i in range(49)]),
                        np.array([i for i in range(49)]),
                        np.array([i for i in range(49)])
                       ])
    ens_num=ens_help[ens_num]
    if ens_num==0:
        return compute_displacement(lat, lon)
    if ens_num==1:
        return compute_displacement(lat, lon, dist=1, bear=90)
    if ens_num==2:
        return compute_displacement(lat, lon, dist=2, bear=90)
    if ens_num==3:
        return compute_displacement(lat, lon, dist=3, bear=90)
    if ens_num==4:
        return compute_displacement(lat, lon, dist=1, bear=270)
    if ens_num==5:
        return compute_displacement(lat, lon, dist=2, bear=270)
    if ens_num==6:
        return compute_displacement(lat, lon, dist=3, bear=270)
    if ens_num==7:
        return compute_displacement(lat, lon, dist=1, bear=180)
    if ens_num==8:
        return compute_displacement(lat, lon, dist=2, bear=180)
    if ens_num==9:
        return compute_displacement(lat, lon, dist=3, bear=180)
    if ens_num==10:
        return compute_displacement(lat, lon, dist=1, bear=0)
    if ens_num==11:
        return compute_displacement(lat, lon, dist=2, bear=0)
    if ens_num==12:
        return compute_displacement(lat, lon, dist=3, bear=0)
    if ens_num==13:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==14:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==15:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==16:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==17:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==18:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==19:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==20:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==21:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==22:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==23:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==24:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==25:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==26:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==27:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==28:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==29:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==30:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==31:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==32:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==33:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==34:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==35:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==36:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==37:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==38:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==39:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==40:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==41:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==42:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==43:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==44:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==45:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==46:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==47:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==48:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)

In [9]:
def compute_displacement(lat, lon, dist=None, bear=None):
    
    """Compute the latitude and longitude for the respective ensemble member.
    
    """
    if not dist:
        return lat, lon
    if dist:
        R = 6378.1 #Radius of the Earth (km)
        brng = math.radians(bear) #Bearing is 90 degrees converted to radians.
        d = dist #Distance in km
        lat1 = math.radians(lat) #Current lat point converted to radians
        lon1 = math.radians(lon) #Current long point converted to radians
        lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
               math.cos(lat1)*math.sin(d/R)*math.cos(brng))
        lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
               math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
        lat2 = math.degrees(lat2)
        lon2 = math.degrees(lon2)
        return lat2, lon2

In [10]:
def height_generator(ens_num, altitude):
    
    """Generate the height for the respective ensemble member.
    
    """
    fraction=np.hstack([np.array([1 for i in range(49)]),
                        np.array([0.95 for i in range(49)]),
                        np.array([0.9 for i in range(49)]),
                        np.array([0.85 for i in range(49)]),
                        np.array([0.8 for i in range(49)]),
                        np.array([0.75 for i in range(49)]),
                        np.array([0.7 for i in range(49)])
                       ])
    return altitude*fraction[ens_num]

In [11]:
which_working


'work1_1'

In [16]:
############################################################################
############################################################################

#where is hysplit working folder?
working_dir = f'/glade/scratch/molina/hysplit/trunk/{which_working}' 
#where is arl format meteo data?
meteo_dir = f'/glade/scratch/molina/basile/{which_climate}'
#where is hysplit model executable?
hysplit_dir=r'/glade/scratch/molina/hysplit/trunk/exec/hyts_std'
#where to put trajs?
output_dir=f'/glade/scratch/molina/basile/{which_climate}_traj/'

############################################################################
############################################################################

runtime = -240

basename = []
years = []
months = []
hours = []
location = []
altitudes = []

#66.736100	-151.93335	261.719970

for i in range(len(csv_file)):

    print(i)
    basename = 'trajid'+str(csv_file.iloc[i][0])+'_subregion'+str(csv_file.iloc[i][2])+'_'+'ens'+str(ens_number)+'_'

    years = [2005]        
    months = [5]
    hours = [6]
    location = (66.7, -151.9)
    altitudes = [height_generator(ens_num=ens_number, altitude=csv_file.iloc[i][6])]
    day1 = (csv_file.iloc[i][10]-1)
    day2 = csv_file.iloc[i][10]
    
    #break

    generate_bulktraj(basename=basename, 
                      hysplit_working=working_dir, 
                      output_dir=output_dir, 
                      meteo_dir=meteo_dir, 
                      years=years, 
                      months=months, 
                      hours=hours, 
                      altitudes=altitudes, 
                      coordinates=location, 
                      run=runtime,
                      meteoyr_2digits=False, outputyr_2digits=False,         
                      monthslice=slice(day1, day2, 1),
                      meteo_bookends=([1] , [1]),
                      get_reverse=False, get_clipped=False, 
                      hysplit=hysplit_dir)

    break

############################################################################
############################################################################
############################################################################ 

0


In [195]:
working_dir

'/glade/scratch/molina/hysplit/trunk/work1_1'

In [196]:
years

[2005]

In [147]:
day1

15

In [76]:
'trajid11_subregion6_ens1_jul731.13293summer2003071700'
'trajid11_subregion6_ens1_jul731.13293summer2003071700
 trajid95_subregion3_ens1_may678.9016spring2005051606
 trajid26_subregion3_ens1_aug420.9251summer2003081206

'trajid11_subregion6_ens1_jul731.13293summer2003071700'

In [145]:
ens_create(ens_num=ens_number, lat=csv_file.iloc[i][4], lon=csv_file.iloc[i][5])

(66.73609836197323, -151.91060579511023)

In [141]:
round(height_generator(ens_num=ens_number, altitude=csv_file.iloc[i][6]),1)

261.7

In [38]:
basename

'trajid26_subregion3_ens1_'

In [199]:
csv_file

,TRAJ. ID,STORM ID,SUBREGION,TIME (UTC),LAT,LON,PBL HEIGHT (M),TIME (INTEGER),YEAR,MONTH,DAY,HOUR
131,132,200508006,1,2005-08-09_00:00:00,68.27313,-164.77606,927.81190,192,2005,8,9,0
132,133,200508020,6,2005-08-24_00:00:00,61.69652,-143.22086,88.05014,552,2005,8,24,0
133,134,200508020,6,2005-08-24_06:00:00,61.85058,-143.41031,872.14240,558,2005,8,24,6
134,135,200508020,6,2005-08-24_12:00:00,61.81105,-143.34400,701.34924,564,2005,8,24,12
135,136,200508021,3,2005-08-23_18:00:00,62.72874,-151.95311,866.35670,546,2005,8,23,18


In [102]:
csv_file

,TRAJ. ID,STORM ID,SUBREGION,TIME (UTC),LAT,LON,PBL HEIGHT (M),TIME (INTEGER),YEAR,MONTH,DAY,HOUR
94,95,200505005,3,2005-05-16_06:00:00,65.824066,-152.21243,678.901600,366,2005,5,16,6
95,96,200505011,3,2005-05-22_06:00:00,62.339940,-155.39578,143.334230,510,2005,5,22,6
96,97,200505013,6,2005-05-15_12:00:00,63.869007,-147.54984,103.526300,348,2005,5,15,12
97,98,200505019,3,2005-05-21_06:00:00,62.036186,-157.70923,315.010600,486,2005,5,21,6
185,186,200705002,3,2007-05-28_00:00:00,65.740520,-152.82231,288.314030,648,2007,5,28,0
186,187,200705002,3,2007-05-28_06:00:00,65.911780,-153.28769,268.879030,654,2007,5,28,6
187,188,200705004,6,2007-05-17_06:00:00,62.062943,-144.04767,511.496830,390,2007,5,17,6
279,280,200905001,3,2009-05-24_06:00:00,64.676820,-157.10298,252.134780,558,2009,5,24,6
280,281,200905007,3,2009-05-23_06:00:00,66.918976,-155.94968,747.120360,534,2009,5,23,6
306,307,201005009,4,2010-05-30_06:00:00,68.884140,-157.23715,205.684200,702,2010,5,30,6
